In [38]:
import spacy
import imaplib
import email
import openpyxl #Lib for excel files
import re
import time
# import nltk
from bs4 import BeautifulSoup

In [ ]:
# from google.colab import drive
# drive.mount('/gdrive')
# %cd /gdrive

In [45]:

host = 'outlook.office365.com'
username = "primetech.pm@outlook.com"
password = "primetech123"

mail = imaplib.IMAP4_SSL(host)
login=False
try:
    mail.login(username,password)
    print("Connected to Mail: "+username)
    login=True
except:
    print("Login Failed")


wb = openpyxl.Workbook()
wb.save('Details.xlsx')
del wb

email_list=[]
contact_list=[]
name_list=[]

my_path = "Details.xlsx" #Enter your excel file name
my_wb_obj = openpyxl.load_workbook(my_path)
my_sheet_obj = my_wb_obj.active
xl = False
try:
    my_wb_obj.save(my_path)
    my_sheet_obj.cell(row=1,column=1).value = "Name"
    my_wb_obj.save(my_path)
    my_sheet_obj.cell(row=1,column=2).value = "Email ID"
    my_wb_obj.save(my_path)
    my_sheet_obj.cell(row=1,column=3).value = "Contact No."
    my_wb_obj.save(my_path)
    xl = True
except Exception as e:
    print("\nClose the Excel Sheet opened in other tabs \nError: ", e)


nlp = spacy.load("en_core_web_sm")

def get_inbox():
    mail.select('INBOX')
    _, data = mail.uid('search',None,"ALL")
    inbox_items = data[0].split()
    messages=[]
    html_messages=[]
    for num in inbox_items:
        _, email_data = mail.uid('fetch', num, '(RFC822)')
        raw_email = email_data[0][1].decode("utf-8")
        email_message = email.message_from_string(raw_email)
        # print(dir(email_message))
        # print(email_message.walk())

        for part in email_message.walk():    
            content_type = part.get_content_type()
            if "plain" in content_type:
              body = part.get_payload(decode=True)
              try:
                messages.append(body.decode("utf-8").replace("\r\n"," "))
              except:
                pass

            # elif "text/html" in content_type:
            #     html_body = part.get_payload()
            #     soup = BeautifulSoup(html_body,'html.parser')
            #     text = soup.get_text()
            #     html_messages.append(text.replace("=\r\n",""))

    return messages

def logout():
    mail.close()
    mail.logout()
    print("Logged Out")


filter = ['0','1','2','3','4','5','6','7','8','9','@','#',"_","/"]
common_words =["My","I","I'm","Name","Contact","Email","Address","Sent","To","From","Date","Signature","Hello","Hey","Hi","Good","Morning","Evening","Afternoon","Founder","Developer","Co-Developer","Co Developer","Director","Project","Manager","Sales","Advisor","Technology","Consultant","Group","Corporation","Thanks","Regards","Dear","Sir"]


def filter_name(inp,filter):
  
  flag=0
  for i in range(len(inp)):
    for j in range(len(filter)):
      if filter[j] in inp[i]:
        return False
  return True


def named(msg):
  doc = nlp(msg)
  names={}
  name=[]
  phone=[]
  mail=[]
  name_flag=0
  name_string=""
  phones={}
  mails={}
  for token in doc:
    if (token.like_num and len(token.text)>=7):
      phones[token.text] = 1
    if (token.like_email):
      mails[token.text] = 1

  for i in range (len(doc)-1,-1,-1):
    if (doc[i].like_num and doc[i].head.like_num and len(doc[i])>=7) and len(doc[i].head)>=7:
      phones[doc[i].head.text]+=phones[doc[i].text]
    if (doc[i].like_email and doc[i].head.like_email):
      mails[doc[i].head.text]+=mails[doc[i].text]


  for token in doc:
    if (token.pos_ == "PROPN" and token.text not in common_words and token.tag_=="NNP" and not token.like_num and not token.like_email or token.tag_=="NNPS"):
      name_flag=1
      name_string+=" "+token.text
    elif (name_flag==1):
      if (filter_name(name_string[1:],filter)):
        names[name_string[1:]]={'email':None, 'phone':None}
        name.append(name_string[1:])
        name_string=""
        name_flag=0
      else:
        name_string=""
        name_flag=0
        
    
    if (token.like_num and len(token.text)>=7):
      ph_flag=0
      for key, val in names.items():
          if name[-phones[token.text]] in key:
            ph_flag=1
            names[key]['phone'] = token.text
            break
      if(ph_flag==0):
        names[name[-phones[token.text]]]['phone'] = token.text
      
    if (token.like_email):
      em_flag=0
      for key, val in names.items():
          if name[-mails[token.text]] in key:
            em_flag=1
            names[key]['email'] = token.text
            break
      if(em_flag==0):
        names[name[-mails[token.text]]]['email'] = token.text
  

  print("------------------- Final Contact Details -------------------")
  print(names)
  i=my_sheet_obj.max_row+1
  for name in names:
    my_sheet_obj.cell(row=i,column=1).value = name
    my_wb_obj.save(my_path)
    my_sheet_obj.cell(row=i,column=2).value = names[name]['email']
    my_wb_obj.save(my_path)
    my_sheet_obj.cell(row=i,column=3).value = names[name]['phone']
    my_wb_obj.save(my_path)
    i=i+1


if __name__ == "__main__":
    if login:
      t1 = time.time()
      messages = get_inbox()
      logout()
      t2 = time.time()
      print("Time for login and read mails: ",(t2-t1),"seconds")
      count=1
      t3 = time.time()
      for message in messages:
        print("--------------------------------------------\n",count,message)
        count+=1
        named(message)
        if(count>20):
          break
      t4 = time.time()
      print("Time for categorization and saving all fields in the Excel Sheet: ",(t4-t3),"seconds")
      print("Total time required: ",(t4-t1)," seconds")
      print("Completed")


Connected to Mail: primetech.pm@outlook.com
Logged Out
Time for login and read mails:  5.1456286907196045 seconds
--------------------------------------------
 1 Hi, I am Priyam Shah. I would like to congratualate you on being selected for the internship. This six months will be really good learning experience for you. Your assigned Project Manager is Mihir Shah. You can contact him by email mihir.ss@somaiya.edu or can call on 9234567890.     Thanks regards, Priyam Shah priyamshah112@gmail.com 7738478888 
------------------- Final Contact Details -------------------
{'Priyam Shah': {'email': 'priyamshah112@gmail.com', 'phone': '7738478888'}, 'Mihir Shah': {'email': 'mihir.ss@somaiya.edu', 'phone': '9234567890'}}
--------------------------------------------
 2 I would like to connect you with my boss, Mr Ravi Shankar and my senior Mr Ayush. You can connect to them via ceo@gmail.com or ravi@gmail.com.     Yours truly, Priyam Shah 7738478888 
------------------- Final Contact Details ----